In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pandas_datareader.data as web
import time
import tensorflow
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import logging
import math
import os
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras.models import load_model

Using TensorFlow backend.


In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

In [3]:
total = pd.read_excel('./data/corr/total_재무지표.xlsx')

In [4]:
total = total[total.columns[:374]]

In [5]:
nc = total[total['회사명'] == '(주)엔씨소프트']
sdi = total[total['회사명'] == '삼성SDI(주)']
ssem = total[total['회사명'] == '삼성전기(주)']
hynix = total[total['회사명'] == '에스케이하이닉스(주)']

# 재무지표 로드

In [6]:
nc = nc.dropna(axis = 1)
sdi = sdi.dropna(axis = 1)
ssem = ssem.dropna(axis = 1)
hynix = hynix.dropna(axis = 1)

In [7]:
nc.sort_values('회계년도', ascending= True, inplace = True)
nc.reset_index(drop = True, inplace = True)
nc = nc.loc[5:,:]
nc.reset_index(drop = True, inplace = True)

In [8]:
sdi.sort_values('회계년도', ascending= True, inplace = True)
sdi.reset_index(drop = True, inplace = True)
sdi = sdi.loc[2:,:]
sdi.reset_index(drop = True, inplace = True)

In [9]:
ssem.sort_values('회계년도', ascending= True, inplace = True)
ssem.reset_index(drop = True, inplace = True)
ssem = ssem.loc[2:,:]
ssem.reset_index(drop = True, inplace = True)

In [10]:
hynix.sort_values('회계년도', ascending= True, inplace = True)
hynix.reset_index(drop = True, inplace = True)
hynix = hynix.loc[3:,:]
hynix.reset_index(drop = True, inplace = True)

# 재무비율 로드

In [11]:
nc_ratio = pd.read_excel('./data/corr/nc_total.xlsx')
sdi_ratio = pd.read_excel('./data/corr/sdi_total.xlsx')
ssem_ratio = pd.read_excel('./data/corr/삼성전기_total.xlsx')
hynix_ratio = pd.read_excel('./data/corr/하이닉스_total.xlsx')

In [12]:
nc_ratio = nc_ratio.dropna(axis = 1)
sdi_ratio = sdi_ratio.dropna(axis = 1)
ssem_ratio = ssem_ratio.dropna(axis = 1)
hynix_ratio = hynix_ratio.dropna(axis = 1)

In [13]:
nc_ratio.corr()['종가'].sort_values(ascending = False)

종가                               1.000000
산술평균                             0.998191
최저가                              0.995621
최고가                              0.993931
[공통]유보율(IFRS연결)                  0.917111
거래년도(*)                          0.821830
[공통]매출액증가율(IFRS연결)               0.703820
[공통]1주당매출액(IFRS연결)               0.625445
[공통]1주당정상영업이익(IFRS연결)            0.613842
[공통]차입금의존도(IFRS연결)               0.607227
[공통]차입금비율(IFRS연결)                0.605129
[공통]부가가치(IFRS연결)                 0.578461
[공통]자본금회전률(IFRS연결)               0.569232
[공통]자본금정상영업이익률(IFRS연결)           0.560964
거래대금                             0.556548
[공통]총자본증가율(IFRS연결)               0.551842
[공통]1주당순이익(IFRS연결)               0.525234
[공통]자기자본증가율(IFRS연결)              0.504292
[공통]자본금순이익률(IFRS연결)              0.498383
[공통]총포괄이익증가율(IFRS연결)             0.481992
[공통]1주당 CASH FLOW(IFRS연결)        0.434028
[공통]정상영업이익증가율(IFRS연결)            0.422587
[공통]세금과공과 대 총비용비율(IFRS연결)        0.392516
[공통]순이익증가율(IFRS연결)               0

# 재무지표와 비율 하나의 데이터프레임으로 merge

### cf. 재무비율은 6개월 단위라  지표도 6개월 단위로 계산

In [14]:
nc = pd.merge(nc_ratio, nc, how = 'left', on = ['회사명', '거래소코드','회계년도'])
sdi = pd.merge(sdi_ratio, sdi, how = 'left', on = ['회사명', '거래소코드','회계년도'])
ssem = pd.merge(ssem_ratio, ssem, how = 'left', on = ['회사명', '거래소코드','회계년도'])
hynix = pd.merge(hynix_ratio, hynix, how = 'left', on = ['회사명', '거래소코드','회계년도'])

# 상관관계 만들기

In [15]:
nc_corr = nc.corr(method = 'pearson')
df_nc = pd.DataFrame(nc_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_nc.columns = ['지표','종가']

sdi_corr = sdi.corr(method = 'pearson')
df_sdi = pd.DataFrame(sdi_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_sdi.columns = ['지표','종가']

ssem_corr = ssem.corr(method = 'pearson')
df_ssem = pd.DataFrame(ssem_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_ssem.columns = ['지표','종가']

hynix_corr = hynix.corr(method = 'pearson')
df_hynix = pd.DataFrame(hynix_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_hynix.columns = ['지표','종가']

In [16]:
list_corr = []
order = 15

for i in range(len(df_nc.index[:order])):
    list_corr.append(df_nc.index[:order][i])
    list_corr.append(df_sdi.index[:order][i])
    list_corr.append(df_ssem.index[:order][i])
    list_corr.append(df_hynix.index[:order][i])

variable = pd.Series(list_corr).value_counts()
variable = pd.DataFrame(variable).reset_index()
variable.columns = ['var_index', 'number']

In [17]:
var = []
for i in range(len(variable)):
    var.append(df_nc.reset_index()['지표'][variable.index[i]])
var = pd.DataFrame(var)
var.columns = ['var']

In [18]:
corr = pd.concat([variable,var], axis = 1)
corr = corr[['var_index','var','number']]
corr.head(20) # 52번까지 재무비율

,var_index,var,number
0,46,종가,4
1,44,산술평균,4
2,45,최저가,4
3,47,최고가,4
4,107,매도가능금융자산평가손익(IFRS연결)(천원),3
5,14,기말기타포괄손익누계액(*)(IFRS연결)(천원),2
6,10,기타포괄손익누계액(*)(IFRS연결)(천원),2
7,49,부채(*)(IFRS연결)(천원),2
8,50,[공통]유보율(IFRS연결),2
9,51,자산(*)(IFRS연결)(천원),2


# 주가와 높은 상관관계를 가진 변수간 상관관계 보기

In [19]:
nc_corr[corr['var'][:20]]

,종가,산술평균,최저가,최고가,매도가능금융자산평가손익(IFRS연결)(천원),기말기타포괄손익누계액(*)(IFRS연결)(천원),기타포괄손익누계액(*)(IFRS연결)(천원),부채(*)(IFRS연결)(천원),[공통]유보율(IFRS연결),자산(*)(IFRS연결)(천원),지배기업주주지분(*)(IFRS연결)(천원),합계(IFRS연결)(천원),자본(*)(IFRS연결)(천원),주식발행초과금(IFRS연결)(천원),미처분이익잉여금(결손금)(IFRS연결)(천원),이익잉여금(결손금)(*)(IFRS연결)(천원),기말이익잉여금(결손금)(*)(IFRS연결)(천원),투자활동으로 인한 현금유입액(*)(IFRS연결)(천원),처분후 이익잉여금(결손금)(*)(IFRS연결)(천원),거래년도(*)
거래소코드,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[공통]총자본증가율(IFRS연결),0.551842,0.568073,0.584981,0.591882,0.578305,0.576701,0.576701,0.596786,0.547468,0.602045,0.594174,0.597626,0.597626,0.494166,0.465282,0.465108,0.465108,0.541804,0.398037,0.389482
[공통]자기자본증가율(IFRS연결),0.504292,0.518479,0.525150,0.543287,0.539896,0.536917,0.536917,0.501974,0.495024,0.543715,0.545611,0.549429,0.549429,0.504292,0.415542,0.415430,0.415430,0.459723,0.378859,0.362579
[공통]매출액증가율(IFRS연결),0.703820,0.688590,0.726282,0.676549,0.747417,0.748201,0.748201,0.791744,0.766343,0.763958,0.749500,0.749311,0.749311,0.584209,0.786314,0.785961,0.785961,0.841087,0.655408,0.602886
[공통]정상영업이익증가율(IFRS연결),0.422587,0.384723,0.428052,0.351313,0.421501,0.420609,0.420609,0.450431,0.548889,0.524062,0.538300,0.538331,0.538331,0.427027,0.633998,0.633834,0.633834,0.627270,0.597226,0.501164
[공통]순이익증가율(IFRS연결),0.361150,0.323705,0.361965,0.292144,0.369571,0.368923,0.368923,0.417666,0.522269,0.492678,0.508053,0.507611,0.507611,0.387596,0.617718,0.617522,0.617522,0.598253,0.582122,0.492202
[공통]총포괄이익증가율(IFRS연결),0.481992,0.513144,0.460476,0.546270,0.488207,0.488813,0.488813,0.469081,0.359264,0.358229,0.328191,0.325698,0.325698,0.348071,0.266345,0.266254,0.266254,0.233203,0.192976,0.291682
[공통]매출액정상영업이익률(IFRS연결),0.094102,0.054460,0.112545,0.018566,0.023292,0.021263,0.021263,0.284501,0.297979,0.294009,0.294704,0.293672,0.293672,0.180678,0.450822,0.451039,0.451039,0.358844,0.327599,0.301190
[공통]매출액순이익률(IFRS연결),-0.107064,-0.143555,-0.096079,-0.169205,-0.185457,-0.186657,-0.186657,0.130305,0.107485,0.098813,0.092538,0.089599,0.089599,-0.061588,0.262808,0.262862,0.262862,0.173324,0.135839,0.139987
[공통]총자본사업이익률(IFRS연결),0.005307,0.005384,0.053631,-0.005722,0.036422,0.039152,0.039152,0.073559,-0.035890,-0.022062,-0.048179,-0.046936,-0.046936,-0.181383,0.021696,0.021623,0.021623,0.162338,-0.224582,-0.243443


# 상관관계 시각화

In [20]:
# import matplotlib.pyplot as plt
# from matplotlib import font_manager, rc
# import seaborn as sns
# import matplotlib
# font_location = "c:/Windows/fonts/malgun.ttf"
# font_name = font_manager.FontProperties(fname=font_location).get_name()
# matplotlib.rc('font', family=font_name)

In [21]:
# plt.figure(figsize=(20,10))

# sns.heatmap(data = nc_corr[corr['var'][:20]], annot = True, cmap = 'bwr')
# # plt.savefig('tmp.png')

# 딥러닝 모델 구축

In [22]:
# 재무피쳐값 입력
corr.loc[1:13,'var']

1                           산술평균
2                            최저가
3                            최고가
4       매도가능금융자산평가손익(IFRS연결)(천원)
5     기말기타포괄손익누계액(*)(IFRS연결)(천원)
6       기타포괄손익누계액(*)(IFRS연결)(천원)
7              부채(*)(IFRS연결)(천원)
8                [공통]유보율(IFRS연결)
9              자산(*)(IFRS연결)(천원)
10       지배기업주주지분(*)(IFRS연결)(천원)
11                합계(IFRS연결)(천원)
12             자본(*)(IFRS연결)(천원)
13           주식발행초과금(IFRS연결)(천원)
Name: var, dtype: object

In [23]:
# 재무 피쳐값 입력 및 종가 입력
nc_deep = nc[corr.loc[1:13,'var']]

nc_deep['종가'] = nc['종가']

In [24]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

# minmax_scale(nc_deep)
nc_deep = pd.DataFrame(scale(nc_deep))

# 샤비어함수 및 히든레이어 5개

In [25]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    if uniform:
        # 6 was used in the paper.
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        # 3 gives us approximately the same limints as above since this repicks
        # values greater than 2 standard deviations from the mean.
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [40]:
nc_deep.iloc[:-1,:-1]

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-1.011820,-1.014623,-0.928666,-0.592763,-0.570595,-0.570595,-1.009153,-1.211916,-1.206814,-1.249767,-1.247046,-1.247046,-1.846755
1,-0.970414,-0.917402,-0.996293,-0.598692,-0.576161,-0.576161,-0.839917,-1.090012,-1.068624,-1.121914,-1.118331,-1.118331,-0.894551
2,-0.091558,0.009055,-0.170276,-0.600092,-0.608377,-0.608377,-0.841954,-0.972030,-0.974256,-0.999551,-0.999581,-0.999581,-0.894551
3,-0.781589,-0.803025,-0.817564,-0.603251,-0.619113,-0.619113,-0.890297,-0.838742,-0.874596,-0.859778,-0.862054,-0.862054,-0.149802
4,-0.782281,-0.854495,-0.764429,-0.603635,-0.618046,-0.618046,-0.825074,-0.598516,-0.660782,-0.609360,-0.611316,-0.611316,-0.149802
5,-0.606275,-0.597145,-0.527734,-0.602919,-0.609787,-0.609787,-0.319616,-0.183593,-0.059059,0.002141,0.009865,0.009865,-0.149802
6,-0.346430,-0.265451,-0.363496,-0.593817,-0.593453,-0.593453,-0.094534,-0.051415,0.083039,0.118128,0.128824,0.128824,-0.149802
7,-0.223547,-0.191106,-0.266886,-0.596021,-0.594263,-0.594263,0.332452,0.091118,0.064444,0.010631,-0.006487,-0.006487,-0.149802
8,-0.008747,-0.133917,0.013284,-0.298853,-0.298800,-0.298800,0.144585,0.393613,0.285450,0.324564,0.319656,0.319656,-0.149802
9,1.462843,1.210018,1.534895,1.495204,1.494179,1.494179,0.673495,0.911394,0.829040,0.863823,0.861861,0.861861,1.511556


In [26]:
# Review : Learning rate and Evaluation
import tensorflow as tf
import random
import matplotlib.pyplot as plt

tf.reset_default_graph()


x_data = nc_deep.iloc[:-1,:-1]
y_data = nc_deep.iloc[:-1,-1] # 괄호를 한번 더 씌어야 한다는 거 주의!!!!!!

print(y_data)
tf.set_random_seed(777)  # reproducibility
placeholder_num = len(x_data.columns)

# parameters
learning_rate = 0.001



X = tf.placeholder(tf.float32, [None, placeholder_num])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)


W1 = tf.get_variable("W1", shape=[placeholder_num, 32], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([32]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)


W2 = tf.get_variable("W2", shape=[32, 64], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[64, 64], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([64]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)


W4 = tf.get_variable("W4", shape=[64, 64], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([64]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)


W5 = tf.get_variable("W5", shape=[64, 1], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([1]))
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)

hypothesis = tf.matmul(L4, W5) + b5


# define cost/loss & optimizer
cost = tf.reduce_mean(tf.square(hypothesis - Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)


# correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

0    -1.099452
1    -0.974215
2    -0.032217
3    -0.750967
4    -0.756412
5    -0.576725
6    -0.418817
7    -0.168344
8    -0.043107
9     1.383504
10    2.134925
Name: 13, dtype: float64


In [38]:
# Launch the graph in a session.
sess = tf.Session()

# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

       
for step in tqdm_notebook(range(2000001)):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train],
                                   feed_dict={X: x_data, Y: y_data, keep_prob: 1.0})

    if step % 10000 == 0 or step < 10 :
        print("\nStep : {} \nCost : {} \nPrediction :\n{}".format(step, cost_val, hy_val))
       

ValueError: Cannot feed value of shape (11,) for Tensor 'Placeholder_1:0', which has shape '(?, 1)'

실제값
0  -1.099452
1  -0.974215
2  -0.032217
3  -0.750967
4  -0.756412
5  -0.576725
6  -0.418817
7  -0.168344
8  -0.043107
9   1.383504
10  2.134925
11  1.301828

Step : 1 
Cost : 8.496781349182129 
Prediction :
[[-2.704463 ]
 [-2.545785 ]
 [-2.6568465]
 [-2.734706 ]
 [-2.61222  ]
 [-2.9015188]
 [-2.7785816]
 [-2.6802928]
 [-2.6181746]
 [-3.0658998]
 [-2.558342 ]
 [-2.8103006]]
 
Step : 10000 
Cost : 0.7597538828849792 
Prediction :
[[-0.0910393 ]
 [-0.03625989]
 [-0.10393304]
 [-0.1150068 ]
 [ 0.03890622]
 [-0.2667489 ]
 [-0.14521682]
 [-0.10149986]
 [-0.0137009 ]
 [ 0.13332021]
 [ 0.4574176 ]
 [ 0.02707112]]
 
Step : 100000 
Cost : 0.07895835489034653 
Prediction :
[[-0.7541409 ]
 [-0.8597247 ]
 [-0.7162141 ]
 [-0.4943511 ]
 [-0.34879994]
 [-0.45882875]
 [-0.4853145 ]
 [-0.34539455]
 [-0.2862141 ]
 [ 1.3516657 ]
 [ 2.0651321 ]
 [ 1.3272438 ]]
 

Step : 1000000 
Cost : 0.005604434758424759 
Prediction :
[[-1.038444  ]
 [-0.86944085]
 [-0.24409688]
 [-0.7571636 ]
 [-0.764     ]
 [-0.594654  ]
 [-0.34014827]
 [-0.15721864]
 [-0.07167464]
 [ 1.3858631 ]
 [ 2.1448278 ]
 [ 1.3050723 ]]
 

Step : 2000000 
Cost : 6.114699499448761e-05 
Prediction :
[[-1.0927246 ]
 [-0.9646204 ]
 [-0.05401689]
 [-0.7489762 ]
 [-0.7564514 ]
 [-0.5807341 ]
 [-0.4096833 ]
 [-0.16719031]
 [-0.04703885]
 [ 1.3833759 ]
 [ 2.1358614 ]
 [ 1.302011  ]]
 
실제값
0  -1.099452
1  -0.974215
2  -0.032217
3  -0.750967
4  -0.756412
5  -0.576725
6  -0.418817
7  -0.168344
8  -0.043107
9   1.383504
10  2.134925
11  1.301828

In [36]:
from tqdm import tqdm_notebook